In [53]:
import matplotlib.pyplot as plt
import networkx as nx
import nltk
import numpy as np
import pandas as pd
import re
from collections import Counter, defaultdict
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from nltk.util import ngrams
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud
import math
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

np.set_printoptions(threshold=np.inf)

In [47]:
# Cargamos los datos
df_clean = pd.read_csv('../data/train_clean.csv')

In [61]:
# 2. Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(df_clean['text'], df_clean['label'], test_size=0.9, random_state=777)

In [40]:
X_train.columns

Index([], dtype='object')

In [73]:
# 1. Cargar tu dataframe df_clean con las columnas 'text' y 'label'

# 2. Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(df_clean['text'], df_clean['label'], test_size=0.98, random_state=42)

# 3. Definir una clase personalizada para combinar los clasificadores Naive Bayes y árbol de decisión
class TANClassifier(BaseEstimator, TransformerMixin):
    def __init__(self, nb_clf, dt_clf):
        self.nb_clf = nb_clf
        self.dt_clf = dt_clf
    
    def fit(self, X, y):
        self.nb_clf.fit(X, y)
        X_transformed = self.nb_clf.predict_proba(X)
        self.dt_clf.fit(X_transformed, y)
        return self
    
    def predict(self, X):
        X_transformed = self.nb_clf.predict_proba(X)
        return self.dt_clf.predict(X_transformed)

# 4. Crear un clasificador TAN utilizando un clasificador Naive Bayes y un árbol de decisión
naive_bayes = GaussianNB()
decision_tree = DecisionTreeClassifier(criterion='entropy')
tan_classifier = TANClassifier(naive_bayes, decision_tree)

# 5. Crear una instancia de TfidfVectorizer para convertir las noticias en vectores TF-IDF
vectorizer = TfidfVectorizer()

# 6. Obtener la representación TF-IDF de los datos de entrenamiento
X_train_transformed = vectorizer.fit_transform(X_train)

# 7. Convertir la matriz dispersa a una matriz densa
X_train_transformed = X_train_transformed.toarray()

# 8. Entrenar el clasificador TAN
tan_classifier.fit(X_train_transformed, y_train)

# 9. Obtener la representación TF-IDF de los datos de prueba
X_test_transformed = vectorizer.transform(X_test)

# 10. Convertir la matriz dispersa a una matriz densa
X_test_transformed = X_test_transformed.toarray()

# 11. Realizar predicciones en los datos de prueba
y_pred = tan_classifier.predict(X_test_transformed)

# 12. Calcular la exactitud del clasificador
accuracy = accuracy_score(y_test, y_pred)
print("Exactitud del clasificador TAN:", accuracy)

MemoryError: Unable to allocate 2.25 GiB for an array with shape (20384, 14832) and data type float64